<a href="https://colab.research.google.com/github/dharshriya15/ML-Projects/blob/main/Exercise_Renaming_and_Combining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085958Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D644e4d091e5ebd4516f2994d04f416e0193db61ffcb39884a1336764ede58f1fe6ee657d9428d5da0e9f01817712546792294f38f59fccc6c97c37c647a3b9beca814b4cb69c0902a501406bca92bfbe1d2e2bf70c0861451142cc1e9a8737be9989db2321cf403aa56825a3d0a7424565f90e315b314c28fcc6b00d32d019965f3a608508bc65bfba89b7b0e75f4becb3445ce584d95d7b0f8827e7d468421489554a2d33f7da63fadffe5becf17da79e1611f5bde741bc70ca700d8cc6d04e4a385da62d304da4e3b7034bba8b5d4e34ed4ac0e6c1f8f72f25aca0e5663d13aab399260ca6e3a4dcb6b4d1000be4ee5b5662254dd14a3c7d1dfd224b8c2cf3,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085958Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4381b98e0337124db39a0fa34c2bba7eb11118acf2e34df6cb1694799a6f013dfde308150face89ad1d909cda5729f99434b959595a00519d6faff1f4001cb078ed478442bf9511d3ef45c2541b87a90ddcfa897ec3922fcadcdf7f3f877f7cd7cff845e075b5b105341d217df8e00ecd45544ac8f602e1e33dac3a55672e14362a6b5a18fdad51e845cd7c46c7d1991f81b24a4b17f3508da689cd966cc4dac9eed27611d4679060b52a6b3f81820d56c4d43bc5ffba2b66108a1d7e593b2c75608c0b881e9f4c8d40d1ad5d330813b312ea3cf4d3f84ea01bc48afd138f1d44045f19a56ed7fc2e3dbe356a70539d988d18fa5f726b4d645aa884275d56d76,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085958Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db938d8331124dbdb931ede8a944a4f8ec4dedce13cdc2ac3beb363508d81be2f7a4dd1f82ba175623bb7a743528e9b8a3358fad5674e8771479665474ccf6c620c0f4e8e887ceb4806fabf9f8263f1b6dcff3db3f432b081594697e81cd716f32608dbb755f0734794a5df19bb6e58ca1dec06ed35dba423d75ccdc9aca9dd06f19c1c441cab02be0d3e83e1acbf50d5def9eb8e7b366787610ef8b0af446b4be5921619c813559fd69f4fa474ca3d0a0ec68cf7cc846a0cbb1d68e9aad2e92a5af19fe0bade785b8f5c55c4f3eda13749a9e4a9ce58615ac89b8a26fae339a9d0f43d4b67e4f0d6b88a25989756c18e0dcac911288163ca0096faad14f32dc6,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085958Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D87818022cf05cff52b9499c7257f5ea27a7e0ba66699a0316a9d987ad3549a3a160e8d202076a05866bf4d79cc5f204d82b1830931c4d9cb26e7ac753531b9cc6939b265afde057a1542faef7c64236ecf0733f99ff544b292dcb96c258c15a9e11d4a73a830c9765610acdd35bc7439913fab0aa1bd70ef521d6652243189c80122fc0cec05dbcb629e1d579e1c77beaee0e0701f8b655ea4df5a21da2ef43d724e02b70f2104847b46e84674714f8ed1a273446c588bb2e72846d50a9c074b77ee19d853196f9aa4f3fd6dd7abed4e0ad32b3d1dd4e956eb7fd67bc26a6255c58bf322f0cc684cbbb6ed5e83702c42950da9e816bfa8a2acefeb085526c7d0,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8ff4c2637cea4c2edf6517fba2b850e405865467e7108221f04634ab87aa0bcc41957099ec0b308c0644c4da9d8e0dbdf5b5d39dc04abd0b48fc7cdf8e31fe3f89b21cc5067d00b6861d7fe5af9df64234cbcc784bee4fa02c9b2d1d7c494b3b46a6adfaf62364c16b44ba89b3a872a57eb5f7164269e56ab44274ab1b344cd48fd170ebc871ba20bb65c59cfd760aa1b2ba266efb7d41d70c4215929408b805b354ecc797b53f1a2dec02f9abbd8b4b30d4cfb5f790019c6c96a70a0b485d2b41a8eb96a4438d9ea1c240f91ae53783a6194c78d4e06861fc8fc16345b812ef9264bee6979ce0e3e61b4c119df251dc09a04e4cd68cd1bda45ebae7ae680f2f,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1acf603eee5d3bfca027a6c7f596ce59886e208f167a1b883b3fe3eea6a18d06955abc733f6a413a2f9bd172ccaff1ed879dbbbdce442ce011f1a5e5b5415aa54e26e7fd60e5e561a16ef993765c7eedc8a5f8d3d6a7cb9897f64c6e75704f8628b170d310ec56e89aac8720ce4c9d2d9ee3a8ec9fc2b2a9bb3dcaf8516feb120ef6face7283592150811d34bd1ffe1171eb9ab00248c0265a0092ef2fdb91b025e88a6f57af4ea9bb8afad619e4b3ba43321e63275bca793e23a54c00dada0e7219c134c74195ffc1739d2c1c8ad638e6f1ac5a1261243848dc6e9992b1b2351dc8f86aca01035f1744e9011f3a07f00b4aa45596ecf3e0d5020394029d120c,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1f1e4c50e131ca5e34e0c5ff265ecf472a77b918ba6a5e2c2488e648f70a926a8db74cd004a272a7f5118e14a9e41f1dfa376d69d66fd6019f8953f2e9b7a8a18a5a82687ac8cbc76fe9d155e4d99395669077408766d4001ebfe55e0a7d0f32e4082538a64b0cd678b570052552952207b447b4de19e47be0ee48bbcd03b2022bc378e36df45e1bae30a1567700d2ad72a2acee384edd85e27004763879ba56355714a020da426c5c7cd398adf5e9f29e1223d09809553f5fe40d9b9ebf44e4ae8df039f45a1412602229e81595e30ca3676d4ab3978ada062cf67f74dea9baadec37e48c3858efca6b0efa60646bea98f2600c457a654ae6feb13b032b30be,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9448129611a82659d039b89b57efedf76e10d2c7e2af81905c3bcc4385e25166e41ffdf7454abfa4775c4de7d8f366805b0cb900a2259b18079469984e080992beeb5d88d2ab0ab1221156437f98ebc20a1b25096409c49d28709b3da26d8f8beedf82d25eb62e54b83aebf1593bd36c93c30aaf58ef4b73cf78aaf8706cef8958fcc730e1d7ae33250ac0e3da29934ba904038bd95c3a234c64059688be531de244383dc83686c66710ad3e409b351028b45261cc7ca5a87be8be959e10e5c3cac6f59a9331bf8047b6badca78c660827faa5d3d283899a31905a41eac861e8dd405ce848fa495c5fde85b95c22c6dfcaf648d89d470b9876e958fe25a6c5cd,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D77b9ca9a11fef375325800277fa8aff5db92ba0b614ba25d0cb3d2d4bf9113df3b4c5b27b48c49d584e5939b8189fd94f615e62bbf836fd4dda53d4c9a668c59dfa11e2f496a0ca726c80c77acb73226d7ae071399c77dffd739c2a7ec140cdbf8bcbb5faf263343085a67d4f93dfa16319dd1392cc4b6c2dc47a35c5c0bfef6e000b4866f9a979a2ca9310b9fa4e391a7e3771710e521f04e52e33698472f566bfefaf8ab3abb9e5c6d313511f4771b2610acb46011741928263eefcdc9a68ca03d3924d9327e6b0c838b8183cdb2307de23b47361ab5564bf5a0d5049b4d4ceab4a5a0488584308f877c5130c8a5e0429e008c346ab83bb1d3f46b0eb41f68,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T085959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9101cbf344568217068080838f75305f806a4927a3233a83720e7bdf81d904cafeec21859028ae21401de26790be4d194fae912768af8c0ba80e82273d3288d6a0a44e3ffe25047c1a5fd5506cd0314d1d553cd37299535f14fc88a7e8e6b91f5aa515c8ae0542c3f078f370a03cba6274449a263557e8ad150f4d780cf31f582d2c2b58b887047f9c3075d2c0bcd3b3e4173ccfeef786119068fa3e0da3d45948a34832431b323d21b47d93f61bf5478be59de0ec0d835546d295779194218568603c7314a8474a7967c67dda3bde94b076c173cdf75d72de1fc00fa84649bcac30426d3cb5674372e9ca13fb84746e7fa055d95416fa3838c4b8a55bdb2eec'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/renaming-and-combining).**

---


# Introduction

Run the following cell to load your data and some utility functions.

In [ ]:
import pandas as pd

reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.renaming_and_combining import *
print("Setup complete.")

Setup complete.


# Exercises

View the first several lines of your data by running the cell below:

In [ ]:
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## 1.
`region_1` and `region_2` are pretty uninformative names for locale columns in the dataset. Create a copy of `reviews` with these columns renamed to `region` and `locale`, respectively.

In [ ]:
# Your code here
renamed = reviews.rename(columns={'region_1':'region','region_2':'locale'})

# Check your answer
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
#q1.hint()
#q1.solution()

## 2.
Set the index name in the dataset to `wines`.

In [ ]:
reindexed = reviews.rename_axis('wines')

# Check your answer
q2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# q2.hint()
#q2.solution()

## 3.
The [Things on Reddit](https://www.kaggle.com/residentmario/things-on-reddit/data) dataset includes product links from a selection of top-ranked forums ("subreddits") on reddit.com. Run the cell below to load a dataframe of products mentioned on the */r/gaming* subreddit and another dataframe for products mentioned on the *r//movies* subreddit.

In [ ]:
gaming_products = pd.reCategoricalDtypead_csv("../input/things-on-reddit/top-things/top-things/reddits/g/gaming.csv")
gaming_products['subreddit'] = "r/gaming"
movie_products = pd.read_csv("../input/things-on-reddit/top-things/top-things/reddits/m/movies.csv")
movie_products['subreddit'] = "r/movies"

Create a `DataFrame` of products mentioned on *either* subreddit.

In [ ]:
combined_products = pd.concat([gaming_products,movie_products])

# Check your answer
q3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
#q3.hint()
#q3.solution()

## 4.
The [Powerlifting Database](https://www.kaggle.com/open-powerlifting/powerlifting-database) dataset on Kaggle includes one CSV table for powerlifting meets and a separate one for powerlifting competitors. Run the cell below to load these datasets into dataframes:

In [ ]:
powerlifting_meets = pd.read_csv("../input/powerlifting-database/meets.csv")
powerlifting_competitors = pd.read_csv("../input/powerlifting-database/openpowerlifting.csv")

Both tables include references to a `MeetID`, a unique key for each meet (competition) included in the database. Using this, generate a dataset combining the two tables into one.

In [ ]:
powerlifting_combined =powerlifting_meets.set_index("MeetID").join(powerlifting_competitors.set_index("MeetID"))

# Check your answer
q4.check()

TypeError: 'str' object is not callable

In [ ]:
# # q4.hint()
# q4.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
powerlifting_combined = powerlifting_meets.set_index("MeetID").join(powerlifting_competitors.set_index("MeetID"))
```

# Congratulations!

You've finished the Pandas micro-course.  Many data scientists feel efficiency with Pandas is the most useful and practical skill they have, because it allows you to progress quickly in any project you have.

If you'd like to apply your new skills to examining geospatial data, you're encouraged to check out our **[Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis)** micro-course.

You can also take advantage of your Pandas skills by entering a **[Kaggle Competition](https://www.kaggle.com/competitions)** or by answering a question you find interesting using **[Kaggle Datasets](https://www.kaggle.com/datasets)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*